In [1]:
import pandas as pd
from pandas import DataFrame
import sklearn.metrics
from numpy import unravel_index
import os
import math

In [2]:
def normalized_values(y,dfmax, dfmin):
    a = (y- dfmin) / (dfmax - dfmin)
    return(a)

def apply_norm(normdf, unnormdf, col, dfmax, dfmin, dfmean, dfstd, norm_type):
    normdf[col] = unnormdf.apply(lambda x: normalized_values(x[col], dfmax, dfmin, dfmean, dfstd, norm_type), axis=1)

def dataset_sanity_check(df):
    for c in [cl for cl in df.columns if 'bin' not in cl]:
        print('column %s - max: %s, min : %s, mean: %s, std: %s'%(c, df[c].max(), df[c].min(), df[c].mean(), df[c].std()))

def normalize_dataset(df, d):
    X = DataFrame()
#    if aggrfile and os.path.exists(aggrfile):
#        with open(aggrfile) as aggrf:
#            aggrs = json.loads(aggrf.read())
    for c in df.columns:
        if c in d.keys():
            print(c)
            dfcfloat = df[c].astype('float64')
            #print("Normalize column:%s" % c)
            dfmax = d[c]['max'] if 'max' in d[c] else None
            dfmin = d[c]['min'] if 'min' in d[c] else None
            #print(d[c]['min'],'-',d[c]['max'])
            #dfmean = d[c]['mean'] if 'mean' in aggrs[c] else None
            #dfstd = d[c]['std'] if 'std' in aggrs[c] else None
            X[c] = dfcfloat.apply(lambda x: normalized_values(x, dfmax, dfmin))#, axis=1)
            #dataset_sanity_check(X[[c]])
        else:
            X[c] = df[c]
        X[c] = X[c].round(3)
    return(X)

In [3]:
d = {'aspect': {'max': 359.99800000000016, 'min': -1.0},
 'curvature': {'max': 0.73661599999999994, 'min': -0.76867799999999997},
 'dem': {'max': 2806.1399999999999, 'min': -6.04},
 'dom_vel': {'max': 19.337629295696285, 'min': 0.49528943306704032},
 'evi': {'max': 0.99250000000000005, 'min': -0.19869999999999999},
 'lst_day': {'max': 342.77999999999997, 'min': 251.78},
 'lst_night': {'max': 322.45999999999998, 'min': 254.47999572753903},
 'max_dew_temp': {'max': 300.5537109375, 'min': 266.383056640625},
 'max_temp': {'max': 317.44775390625, 'min': 273.447509765625},
 'mean_dew_temp': {'max': 298.55347696940106, 'min': 263.98918660481769},
 'mean_temp': {'max': 309.74698893229169, 'min': 270.31141153971362},
 'min_dew_temp': {'max': 297.611083984375, 'min': 258.5595703125},
 'min_temp': {'max': 303.00634765625, 'min': 262.879150390625},
 'ndvi_new': {'max': 0.99939999999999996, 'min': -0.20000000000000001},
 'rain_7days': {'max': 3.5451469016037436, 'min': 6.8902961061212417e-06},
 'res_max': {'max': 19.337629295696285, 'min': 2.7861490154313331},
 'slope': {'max': 45.0045, 'min': 0.0}}

In [4]:
#df = pd.read_csv('/work2/pa21/sgirtsou/production/2010/05/may_2010_dummies.csv')

In [10]:
dict = {}
for folder, subfolders, files in os.walk('/work2/pa21/sgirtsou/production', topdown=True):
    for file in files:
        if file.endswith('dummies.csv'):
            if file.split('_')[1] in dict:
                dict[file.split('_')[1]].append(os.path.join(os.path.abspath(folder), file))
            else:
                dict[file.split('_')[1]]=[]
                dict[file.split('_')[1]].append(os.path.join(os.path.abspath(folder), file))

{'2010': ['/work2/pa21/sgirtsou/production/2010/05/may_2010_dummies.csv',
  '/work2/pa21/sgirtsou/production/2010/08/august_2010_dummies.csv',
  '/work2/pa21/sgirtsou/production/2010/09/september_2010_dummies.csv',
  '/work2/pa21/sgirtsou/production/2010/04/april_2010_dummies.csv',
  '/work2/pa21/sgirtsou/production/2010/07/july_2010_dummies.csv',
  '/work2/pa21/sgirtsou/production/2010/06/june_2010_dummies.csv'],
 '2011': ['/work2/pa21/sgirtsou/production/2011/05/may_2011_dummies.csv',
  '/work2/pa21/sgirtsou/production/2011/08/august_2011_dummies.csv',
  '/work2/pa21/sgirtsou/production/2011/09/september_2011_dummies.csv',
  '/work2/pa21/sgirtsou/production/2011/04/april_2011_dummies.csv',
  '/work2/pa21/sgirtsou/production/2011/07/july_2011_dummies.csv',
  '/work2/pa21/sgirtsou/production/2011/06/june_2011_dummies.csv'],
 '2012': ['/work2/pa21/sgirtsou/production/2012/05/may_2012_dummies.csv',
  '/work2/pa21/sgirtsou/production/2012/08/august_2012_dummies.csv',
  '/work2/pa21/sgirts

In [7]:
dict={'2015': ['/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv',
  '/work2/pa21/sgirtsou/production/2015/08/august_2015_dummies.csv',
  '/work2/pa21/sgirtsou/production/2015/09/september_2015_dummies.csv',
  '/work2/pa21/sgirtsou/production/2015/04/april_2015_dummies.csv',
  '/work2/pa21/sgirtsou/production/2015/07/july_2015_dummies.csv',
  '/work2/pa21/sgirtsou/production/2015/06/june_2015_dummies.csv']}

# Cosine Similarity

In [8]:
columns = ['max_temp', 'min_temp', 'mean_temp',
       'res_max', 'dom_vel', 'rain_7days', 'dem', 'slope', 'curvature',
       'aspect', 'ndvi_new', 'evi', 'lst_day', 'lst_night', 'max_dew_temp',
       'mean_dew_temp', 'min_dew_temp', 'fire', 'dir_max_1', 'dir_max_2',
       'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6', 'dir_max_7',
       'dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3', 'dom_dir_4',
       'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8', 'corine_111',
       'corine_112', 'corine_121', 'corine_122', 'corine_123', 'corine_124',
       'corine_131', 'corine_132', 'corine_133', 'corine_141', 'corine_142',
       'corine_211', 'corine_212', 'corine_213', 'corine_221', 'corine_222',
       'corine_223', 'corine_231', 'corine_241', 'corine_242', 'corine_243',
       'corine_244', 'corine_311', 'corine_312', 'corine_313', 'corine_321',
       'corine_322', 'corine_323', 'corine_324', 'corine_331', 'corine_332',
       'corine_333', 'corine_334', 'corine_411', 'corine_412', 'corine_421',
       'corine_422', 'corine_511', 'corine_512', 'corine_521', 'wkd_0',
       'wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6', 'month_5',
       'month_4', 'month_6', 'month_7', 'month_8', 'month_9']

In [9]:
def cosine_similarity(fires,nofires):
    #fires = df[df.fire == 1][columns]
    n = fires.max_temp.count()
#   print(n,' fires found')
    #nofires = df[df['fire'] == 0][columns]
    #if n == 0:
    #    return 0,0
    #else:
    print('Finding similarity....')
    similarity = sklearn.metrics.pairwise.cosine_similarity(nofires[columns], Y=fires[columns], dense_output=True)
    return n,similarity

In [10]:
def create_dataset(fires,nofires,df1):
    f_list=[]
    #n,similarity = fires.apply(lambda x:cosine_similarity(pd.DataFrame(x),nofires,similarity),axis=1)
    n, similarity = cosine_similarity(fires,nofires)
    print('Creating similarity dataframe')
    sim_df = pd.DataFrame(data = similarity,index=nofires.index,columns=fires.index)
    print('similarity dataframe ok')
    process_sim = sim_df
    ind = list(process_sim.nlargest(n*20, sim_df.columns).index)
    print('Found',n*20,'largest similarities')
    f_list.append((ind)[:])
    print('Appended indexes')
#     for i,column in enumerate(process_sim.columns):
#         print('Finding 20 most similar for column ',i)
#         ind = list(process_sim.nlargest(20, column).index)
#         f_list.append((ind)[:])
#         process_sim = process_sim.drop(ind)
    flat_list = [item for sublist in f_list for item in sublist]
    print('Calculating neg_pos')
    neg_pos = df1.loc[(df1.index.isin(flat_list))]
    print('Calculated neg_pos')
    return neg_pos
# tran = np.transpose(similarity)
# tran_df = pd.DataFrame(data = tran,index=df1[df1.fire==0].index,columns=df[df.fire==1].index)
# tran_df.to_csv('/'+os.path.join(*filename.split('/')[:-1]+'/similarities.csv)

In [11]:
for key in sorted(dict.keys()):
    dataset = DataFrame()
    print('Year:'+key)
    for i, filename in enumerate(dict[key]):
        f_list=[]
        print('Reading ',i, filename)
        df = pd.read_csv(filename)
        try:
            df = df.drop(['Unnamed: 0'],axis=1)
        except:
            print('No Unnamed: 0.1.1 column')
        df = df.fillna(0)
        n_fires = df[df.fire==1]['id'].count()
        print(n_fires,' fires found')
        if n_fires == 0:
            continue
        else:
            df['wkd'] = pd.to_datetime(df['firedate'],format='%Y%m%d').dt.dayofweek
            df['month'] = pd.to_datetime(df['firedate'],format='%Y%m%d').dt.month
            print('Getting the dummies ......')
            df1 = df.fillna(0)
            df1 = pd.get_dummies(df, columns=['wkd', 'month'])
            del df
            months = ['month_4','month_5','month_6','month_7','month_8','month_9']
            for m in months:
                if m not in df1.columns:
                    print(m+ 'was not in columns of' + filename)
                    df1[m] = 0   
            print('Starting normalization....')
            df1 = normalize_dataset(df1,d)
            print('Saving normalized dataset....')
            #df1.to_csv(filename[:-12]+'_norm.csv')
            fires = df1[df1.fire == 1]
            nofires = df1[df1.fire == 0]
            for j in range(0,math.ceil((n_fires)/100)):
                print(j,'/',math.ceil((n_fires)/100))
                fires_=fires.iloc[j*100:(j+1)*100]
                neg_pos = create_dataset(fires_,nofires,df1)
                if i ==0:
                    neg_pos.to_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/balanced_'+key+'_new.csv')
                    i = 1
                else:
                    neg_pos.to_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/balanced_'+key+'_new.csv',mode='a', header=False)
                #dataset = dataset.append(neg_pos,ignore_index=True)
            df1[df1.fire == 1].to_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/balanced_'+key+'_new.csv',mode='a', header=False)
            #dataset = dataset.append(df1[df1.fire == 1],ignore_index=True)
            #print(dataset[dataset.fire==1]['id'].count(),'fires in dataset')
    #dataset.to_csv('/work2/pa21/sgirtsou/production/datasets/hard_cosine_similarity/balanced_'+key+'.csv',mode='a', header=False)
#df1.to_csv(filename[:-12]+'_norm.csv')

Year:2015
Reading  0 /work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
7  fires found
Getting the dummies ......
month_4was not in columns of/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
month_6was not in columns of/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
month_7was not in columns of/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
month_8was not in columns of/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
month_9was not in columns of/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
Starting normalization....
max_temp
min_temp
mean_temp
res_max
dom_vel
rain_7days
dem
slope
curvature
aspect
ndvi_new
evi
lst_day
lst_night
max_dew_temp
mean_dew_temp
min_dew_temp
Saving normalized dataset....
0 / 1
Finding similarity....
Creating similarity dataframe
similarity dataframe ok
Found 140 largest similarities
Appended indexes
Calculating neg_pos
Calculated neg_pos
Reading  1 /work2/pa21/sgirtsou/production/2015/08/

In [24]:
for key in sorted(dict.keys()):
    print('Year:'+key)
    for i, filename in enumerate(dict[key]):
        print('Reading ',i, filename)
        df = pd.read_csv(filename,nrows=1)
        #print(df.columns)
        print(len(df.columns))

Year:2015
Reading  0 /work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv
77
Reading  1 /work2/pa21/sgirtsou/production/2015/08/august_2015_dummies.csv
77
Reading  2 /work2/pa21/sgirtsou/production/2015/09/september_2015_dummies.csv
76
Reading  3 /work2/pa21/sgirtsou/production/2015/04/april_2015_dummies.csv
76
Reading  4 /work2/pa21/sgirtsou/production/2015/07/july_2015_dummies.csv
76
Reading  5 /work2/pa21/sgirtsou/production/2015/06/june_2015_dummies.csv
76


In [29]:
df = pd.read_csv('/work2/pa21/sgirtsou/production/2015/05/may_2015_dummies.csv',nrows=1)

In [31]:
df.columns

Index(['Unnamed: 0', 'id', 'firedate', 'max_temp', 'min_temp', 'mean_temp',
       'res_max', 'dom_vel', 'rain_7days', 'dem', 'slope', 'curvature',
       'aspect', 'ndvi_new', 'evi', 'lst_day', 'lst_night', 'max_dew_temp',
       'mean_dew_temp', 'min_dew_temp', 'fire', 'dir_max_1', 'dir_max_2',
       'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6', 'dir_max_7',
       'dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3', 'dom_dir_4',
       'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8', 'corine_111',
       'corine_112', 'corine_121', 'corine_122', 'corine_123', 'corine_124',
       'corine_131', 'corine_132', 'corine_133', 'corine_141', 'corine_142',
       'corine_211', 'corine_212', 'corine_213', 'corine_221', 'corine_222',
       'corine_223', 'corine_231', 'corine_241', 'corine_242', 'corine_243',
       'corine_244', 'corine_311', 'corine_312', 'corine_313', 'corine_321',
       'corine_322', 'corine_323', 'corine_324', 'corine_331', 'corine_332',
       'corine_333', 'c

In [18]:
df.to_csv('/work2/pa21/sgirtsou/production/2013/06/june_2013_dummies.csv')/work2/pa21/sgirtsou/production/2013/06/june_2013_dummies.csv